# Projeto de Modelagem de Aprovação de Crédito: Um Case

## Contextualização

No início de 2023, o departamento de crédito do Banco Nacional de Finanças (BNF) enfrentava um desafio significativo. Com uma taxa de inadimplência crescente e um processo manual de análise que levava, em média, 3 dias úteis para aprovar ou rejeitar solicitações de crédito, era evidente que uma transformação digital se fazia necessária.

Durante uma reunião trimestral de resultados, Maria Silva, Diretora de Crédito do BNF, apresentou números preocupantes:

- Tempo médio de análise: 3 dias úteis
- Taxa de inadimplência: 12% (2% acima do mercado)
- Taxa de abandono de solicitações: 22% (clientes desistiam durante o processo)
- Alta correlação entre demora para aprovação e abandono de solicitações
- Custo operacional da análise manual: R$2,2 milhões/ano

"Precisamos modernizar nosso processo de análise de crédito. Acredito que a ciência de dados possa nos ajudar a tomar decisões mais precisas e ágeis", afirmou Maria durante a apresentação.

Foi assim que, na semana seguinte, nosso time de Data Science recebeu um e-mail com o assunto: "URGENTE: Projeto Estratégico - Modelo de Aprovação de Crédito Automatizado".

O desafio estava lançado: desenvolver um modelo preditivo capaz de aprovar ou rejeitar solicitações de crédito de forma automática, reduzindo o tempo de análise para segundos, diminuindo a taxa de inadimplência/abandono e, consequentemente, aumentando a rentabilidade da carteira de crédito do banco.

Nossa jornada começou com a coleta e análise de dados históricos de aprovações e rejeições de crédito do último ano. Este notebook documenta o processo de desenvolvimento, avaliação e implementação do modelo que transformaria o departamento de crédito do BNF.

## Objetivos do Projeto

1. Reduzir o tempo de análise de crédito de 3 dias para menos de 1 minuto
2. Diminuir a taxa de inadimplência em pelo menos 3 pontos percentuais
3. Aumentar a aprovação de bons pagadores em 15%
4. Criar um modelo explicável/interpretável que possa ser auditado pelos reguladores bancários

Vamos iniciar esta jornada explorando os dados disponíveis e entendendo os padrões que podem nos ajudar a prever com precisão quais clientes têm maior probabilidade de honrar seus compromissos financeiros.


# 1. Explorando e Tratando os Dados

## 1.1 Importando Bibliotecas

In [1]:
import pandas as pd

## 1.2 Lendo dados e obtendo informações iniciais

In [2]:
df = pd.read_csv("Application_Data.csv")

In [3]:
df.shape

(25128, 21)

In [ ]:
df.head()